In [10]:
import os
import pathlib
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.utils.class_weight import compute_class_weight
seed = 42

os.chdir("../")

pathlib.Path.cwd()

PosixPath('/home/paolo/git')

## Create train test split

In [11]:
def generate_group(x: str) -> str:
    group = re.sub(pattern="_(frame|image)_.*", repl="", string=x.split("/")[-1][:-4])
    return re.sub(pattern="_[0-9]+$", repl="", string=group)

In [12]:
other_animals = [str(x) for x in pathlib.Path("data/bronze/images/other_animals").rglob("*")]
wild_boar = [str(x) for x in pathlib.Path("data/bronze/images/wild_boar").rglob("*")]


data: pd.DataFrame = pd.concat([
    pd.DataFrame(data={"path": wild_boar, "target": [1]*len(wild_boar)}),
    pd.DataFrame(data={"path": other_animals, "target": [0]*len(other_animals)})
]).reset_index(drop=True)
data

,path,target


In [13]:
groups = data["path"].apply(lambda x: generate_group(x))
groups

Series([], Name: path, dtype: float64)

In [14]:
groups.value_counts()

Series([], Name: count, dtype: int64)

In [15]:
try:
    for i in range(2, 200):
        
        sgkf = StratifiedGroupKFold(n_splits=i, shuffle=True, random_state=seed)
        
        train_index, valid_index = next(sgkf.split(X=data.path, y=data.target, groups=groups))
        
        train_size = train_index.shape[0] / data.shape[0]
        print(i, round(train_size, 3))
        if 0.75 <= train_size <= 0.8:
            break
        
except ValueError as e:
    print(e)
    pass

Cannot have number of splits n_splits=2 greater than the number of samples: n_samples=0.


In [16]:
train_data = data.loc[train_index] #.to_parquet(pathlib.Path("../data/train.parquet"), index=False)
valid_data = data.loc[valid_index] #.to_parquet(pathlib.Path("../data/valid.parquet"), index=False)

KeyError: "None of [Index([   1,    2,    4,    5,    6,    7,    8,    9,   10,   11,\n       ...\n       4989, 4990, 4991, 4992, 4993, 4994, 4995, 4996, 4997, 4998],\n      dtype='int64', length=3883)] are in the [index]"

In [ ]:
class_weight = compute_class_weight(class_weight="balanced", classes=np.unique(train_data.target), y=train_data.target)
class_weight = class_weight.astype(np.float32)

In [ ]:
train_data["weight"] = class_weight[train_data.target]
valid_data["weight"] = class_weight[valid_data.target]
train_data["target"] = train_data["target"].astype(np.float32)
valid_data["target"] = valid_data["target"].astype(np.float32)

In [ ]:
train_data

In [ ]:
train_data.to_parquet(pathlib.Path("data/train.parquet"), index=False)
valid_data.to_parquet(pathlib.Path("data/valid.parquet"), index=False)